In [1]:
import pandas as pd
from collections import Counter

file = 'carb-2020-ghg-emissions.csv'
df = pd.read_csv(file)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831 entries, 0 to 830
Data columns (total 20 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   arb_id                                  831 non-null    int64  
 1   facility_name                           831 non-null    object 
 2   reporting_year                          831 non-null    int64  
 3   total_emissions_co2e                    831 non-null    float64
 4   total_emissions_co2e_non_biogenic       831 non-null    float64
 5   co2_biogenic                            831 non-null    float64
 6   fuel_supplier_co2e_non_biogenic         831 non-null    float64
 7   fuel_supplier_co2e_biogenic             831 non-null    float64
 8   electricity_importer_Correspondance2e   831 non-null    float64
 9   emitter_covered_emissions               831 non-null    float64
 10  fuel_supplier_covered_emissions         831 non-null    float6

In [3]:
df.columns

Index(['arb_id', 'facility_name', 'reporting_year', 'total_emissions_co2e',
       'total_emissions_co2e_non_biogenic', 'co2_biogenic',
       'fuel_supplier_co2e_non_biogenic', 'fuel_supplier_co2e_biogenic',
       'electricity_importer_Correspondance2e', 'emitter_covered_emissions',
       'fuel_supplier_covered_emissions',
       'electricity_importer_covered_emissions', 'total_covered_emissions',
       'total_non_covered_emissions', 'city', 'state', 'zip_code', 'naics',
       'epa_carb_subparts', 'industry_sector'],
      dtype='object')

In [4]:
def code(s):
    hyp = s.split('-')
    return int(hyp[0])

def first_n(int_input=None, n=3):
    return int(str(int_input)[:n])

In [5]:
df['naics_code'] = df['naics'].apply(code)
df['first_three_naics'] = df['naics_code'].apply(first_n)
df.head(3)

,arb_id,facility_name,reporting_year,total_emissions_co2e,total_emissions_co2e_non_biogenic,co2_biogenic,fuel_supplier_co2e_non_biogenic,fuel_supplier_co2e_biogenic,electricity_importer_Correspondance2e,emitter_covered_emissions,...,total_covered_emissions,total_non_covered_emissions,city,state,zip_code,naics,epa_carb_subparts,industry_sector,naics_code,first_three_naics
0,5002,Southern California Gas Company - End User Emi...,2020,40409268.21,0.0,0.0,40388206.76,21061.45,0.00,0.0,...,20754129.02,19655139.19,Los Angeles,CA,90013,221210 - Natural Gas Distribution,NN,"Supplier of Natural Gas, NGL, or LPG",221210,221
1,3042,Los Angeles Department of Water & Power (LADWP...,2020,4741327.06,0.0,0.0,0.00,0.00,4741327.06,0.0,...,3313671.67,1427655.39,Los Angeles,CA,90012,221121 - Electric Bulk Power Transmission and ...,EPE,Electricity Importer,221121,221
2,104024,Pacific Gas and Electric Company (PG&E) - Supp...,2020,40304582.67,0.0,0.0,40304582.67,0.00,0.00,0.0,...,17880395.19,22424187.48,San Francisco,CA,94105,221210 - Natural Gas Distribution,NN,"Supplier of Natural Gas, NGL, or LPG",221210,221


In [6]:
# get total emissions grouped by naics_code

In [7]:
sub_list = ['naics_code', 'total_emissions_co2e']

df1 = df[sub_list].groupby(['naics_code'], as_index=False).sum().sort_values(by=['total_emissions_co2e'])
df1.head()

,naics_code,total_emissions_co2e
102,454390,0.00
123,623210,0.00
125,921120,0.00
84,333249,26.53
30,311422,837.34


In [8]:
df2 = df[['naics_code', 'total_emissions_co2e']].groupby(['naics_code'], as_index=False).count().sort_values(by=['total_emissions_co2e'])

In [9]:
df2.rename(columns={'total_emissions_co2e': 'total_facility_count'}, inplace=True)
df2.head()

,naics_code,total_facility_count
89,336111,1
113,541712,1
90,336390,1
111,523110,1
98,447110,1


In [10]:
df2.shape

(132, 2)

In [11]:
df_summary = pd.merge(df1, 
                      df2, 
                      on="naics_code", 
                      how='inner').sort_values(by=['total_emissions_co2e'], 
                                               ascending=False)

In [12]:
df_summary.head()

,naics_code,total_emissions_co2e,total_facility_count
131,221210,90449216.62,22
130,424710,74575779.98,13
129,324110,62552499.89,24
128,221112,33935235.07,131
127,424720,30374602.03,35


In [13]:
df_summary['first_three_naics'] = df_summary['naics_code'].apply(first_n)

In [14]:
covered_ind = df_summary

In [15]:
covered_ind.head()

,naics_code,total_emissions_co2e,total_facility_count,first_three_naics
131,221210,90449216.62,22,221
130,424710,74575779.98,13,424
129,324110,62552499.89,24,324
128,221112,33935235.07,131,221
127,424720,30374602.03,35,424


In [16]:
#df3 = df[sub_list].drop_duplicates(keep='first')
#covered_ind = pd.merge(df_summary, df3, on='naics_code', how='inner')

In [17]:
# NAICS Code 327 – Non-Metallic Mineral Manufacturing
# https://www.calmac.org/publications/Final_Industrial_Glass_Sector_Characterization_Report.pdf
covered_ind[covered_ind['first_three_naics'] == 327]

,naics_code,total_emissions_co2e,total_facility_count,first_three_naics
124,327310,7667665.72,8,327
103,327213,400126.78,4,327
95,327211,201077.37,2,327
93,327420,195648.78,5,327
83,327993,105893.57,4,327
66,327410,64381.43,1,327
47,327123,28824.17,2,327
25,327120,13021.74,1,327


In [18]:
# NAICS 322: Paper Manufacturing
# https://www.calmac.org/publications/Final_Paper_Industrial_Sector_Market_Characterization.pdf
covered_ind[covered_ind['first_three_naics'] == 322]

,naics_code,total_emissions_co2e,total_facility_count,first_three_naics
101,322130,346406.43,4,322
99,322121,337830.63,3,322
15,322299,9527.07,1,322


In [19]:
# Plastics Manufacturing NAICS Codes 325–326 
# https://www.calmac.org/publications/Final_Plastics_Market_Characterization.pdf
covered_ind[(covered_ind['first_three_naics'] == 325) | (covered_ind['first_three_naics'] == 326)]

,naics_code,total_emissions_co2e,total_facility_count,first_three_naics
121,325120,2865062.51,7,325
112,325199,1089290.60,7,325
92,325193,194483.31,4,325
84,325194,112187.44,1,325
79,325412,84433.97,4,325
72,325414,70408.42,4,325
70,325188,69681.59,4,325
51,325180,32160.07,3,325
29,326140,14768.63,1,326
13,325311,9495.74,1,325


In [20]:
# Gas Consumption in the NAICS 311, Basic Metal Producers 
# https://www.calmac.org/publications/Final_metalworking_market_characterization_report.pdf
covered_ind[covered_ind['first_three_naics'] == 311]

,naics_code,total_emissions_co2e,total_facility_count,first_three_naics
109,311421,816971.83,19,311
98,311514,334480.34,11,311
97,311513,264152.66,7,311
89,311423,133347.18,4,311
86,311613,122055.71,7,311
81,311313,91113.21,1,311
80,311919,89301.31,3,311
69,311911,69490.53,4,311
61,311615,57757.59,2,311
60,311511,52908.72,3,311


In [21]:
# (North American Industry Classification System [NAICS] prefix 32731)
# https://www.calmac.org/publications/Final_Cement_Industrial_Market_Characterization_Report.pdf
covered_ind[covered_ind['first_three_naics'] == 327]

,naics_code,total_emissions_co2e,total_facility_count,first_three_naics
124,327310,7667665.72,8,327
103,327213,400126.78,4,327
95,327211,201077.37,2,327
93,327420,195648.78,5,327
83,327993,105893.57,4,327
66,327410,64381.43,1,327
47,327123,28824.17,2,327
25,327120,13021.74,1,327


In [22]:
sub_set = covered_ind[(covered_ind['first_three_naics'] >= 300) & (covered_ind['first_three_naics'] <= 332)]

In [23]:
covered_ind[['naics_code', 'first_three_naics']].groupby(['first_three_naics', 
                                                    'naics_code']).count().head(65)

Empty DataFrame
Columns: []
Index: [(111, 111419), (111, 111422), (211, 211111), (211, 211112), (212, 212299), (212, 212319), (212, 212321), (212, 212391), (212, 212399), (221, 221111), (221, 221112), (221, 221114), (221, 221116), (221, 221117), (221, 221118), (221, 221119), (221, 221121), (221, 221122), (221, 221210), (221, 221310), (221, 221320), (221, 221330), (311, 311111), (311, 311119), (311, 311211), (311, 311223), (311, 311225), (311, 311313), (311, 311411), (311, 311421), (311, 311422), (311, 311423), (311, 311511), (311, 311513), (311, 311514), (311, 311611), (311, 311613), (311, 311615), (311, 311824), (311, 311830), (311, 311911), (311, 311919), (311, 311991), (312, 312111), (312, 312120), (312, 312130), (313, 313310), (321, 321113), (321, 321212), (321, 321219), (322, 322121), (322, 322130), (322, 322299), (324, 324110), (324, 324121), (324, 324122), (324, 324191), (324, 324199), (325, 325120), (325, 325180), (325, 325188), (325, 325193), (325, 325194), (325, 325199), (325, 325311)]

[65 rows x 0 columns]

In [24]:
Counter(sorted(list(covered_ind['first_three_naics'].tolist())))

Counter({111: 2,
         211: 2,
         212: 5,
         221: 13,
         311: 21,
         312: 3,
         313: 1,
         321: 3,
         322: 3,
         324: 5,
         325: 9,
         326: 1,
         327: 8,
         331: 5,
         332: 3,
         333: 3,
         334: 2,
         336: 5,
         424: 4,
         447: 2,
         454: 3,
         481: 1,
         482: 1,
         484: 1,
         486: 2,
         488: 2,
         493: 1,
         523: 2,
         541: 2,
         551: 1,
         562: 2,
         611: 1,
         621: 1,
         622: 3,
         623: 1,
         713: 1,
         921: 2,
         922: 1,
         923: 1,
         924: 1,
         927: 1,
         928: 1})

In [25]:
# Disaggregate these NAICS prefixes.
set(sub_set['first_three_naics'])

# For each, find permit, and get equipment summary (fired equipment)

{311, 312, 313, 321, 322, 324, 325, 326, 327, 331, 332}

In [28]:
covered_ind[covered_ind['first_three_naics'] == 324]

,naics_code,total_emissions_co2e,total_facility_count,first_three_naics
129,324110,62552499.89,24,324
107,324199,489074.33,3,324
49,324121,29877.47,3,324
14,324122,9499.45,1,324
6,324191,1712.49,1,324


In [27]:
covered_ind['total_facility_count'].sum()

831